# Antisense Intron Pipeline
Here we demonstrate how to follow our workflow to identifying antisense intronic smRNAs.
Here we assume that users have already downloaded the raw sequencing data in fastq format or in already aligned data. We use data from GSE 113301 as an example. The code here can be easily written in script form to run in parallel when working with large datasets.

In [1]:
import pandas as pd
import numpy as np

### Util

In [ ]:
def simpleDustScore(seq):
    assert len(seq) > 2
    if len(seq) == 3:
        return 0
    else:
        triplets = {}
        num_trip = len(seq) - 2
        for i in range(num_trip):
            subseq = seq[i:i+3]
            if subseq in triplets:
                triplets[subseq] += 1
            else:
                triplets[subseq] = 1
        sum_triplet = 0
        for triplet, count in triplets.items():
            sum_triplet += count * (count - 1) / 2
        return sum_triplet/(num_trip - 1)

### Step 1: Alignment
We align the files using bowtie2, end-toend, sensitive, with no mismatch allowed. 

In [ ]:
%%bash
for f in data/GSE113301/*.fastq.gz; do
out=${f/.fastq.gz/.bam}
echo "bowtie2 --sensitive --end-to-end -p 12 -x /rumi/shams/jwang/Noelle_Lab/genomes/ws253/ws253 -U $f | samtools view -S -b -h -t /rumi/shams/jwang/Noelle_Lab/genomes/ws253/ws253.fa.fai -o $out";
bowtie2 --sensitive --end-to-end -p 12 -x /rumi/shams/jwang/Noelle_Lab/genomes/ws253/ws253 -U $f | samtools view -S -b -h -t /rumi/shams/jwang/Noelle_Lab/genomes/ws253/ws253.fa.fai -o $out
done &> log/GSE113301/GSE113301.bowtie2.out

In [ ]:
%%bash
#Sort and index our alignment files.
for f in data/GSE113301/*.bam; do
out=${f/.bam/.srt.bam};
echo "samtools sort -@ 4 -m 2G -o $out $f";
samtools sort -@ 4 -m 2G -o $out $f
echo "samtools index $out";
samtools index $out;
done &> log/GSE113301/GSE113301.sort_index.out

### Step 2: Filter Reads
Our filter requirements include:
* MapQ cutoff of 40 to eliminate multimapping reads
* Dust score cutoff of 3 to eliminate low-complexity sequences
* Min read length of 15 (which can also be enforced in the adapter trimming step).

In [ ]:
inputs = [f for f in os.scandir("data/GSE113301") if f.name.endswith("srt.bam")]
len(inputs)

In [ ]:
for f in inputs:
    name = f.name.split(".")[0]
    out = f"data/GSE113301/{name}.srt.fil.bam"
    infile = pysam.AlignmentFile(f, "rb")
    outfile = pysam.AlignmentFile(out, "wb", template=infile)   
    for read in infile.fetch():
        if read.mapq >=40 and simpleDustScore(read.get_forward_sequence()) < 3 and len(read.get_forward_sequence()) >= 15:
            outfile.write(read)
    outfile.close()
    infile.close()

### Step 3: Convert Reads to BED format

In [ ]:
%%bash
proj=GSE113301
for f in data/$proj/*srt.fil.bam; do 
base=$(basename $f)
out=${base/.bam/.bed}
echo "$bedtools bamtobed -i $f > data/bedfiles/$out"
bedtools bamtobed -i $f > data/bedfiles/$out
done &> log/$proj/$proj.bamtobed.out

### Step 4: Antisense Intron Mapping
Using the intron annotation file that we created (in `prepare_intron_map.ipynb`, we then use bedtools intersect to identify reads that map antisense to introns. <br>
The `-S` option enforces overlaps that map to the oppposite/antisense strand.

In [ ]:
%%bash
#Temp for just GSE113301
for f in /rumi/shams/jwang/Noelle_Lab/data/bedfiles/*.srt.fil.bed; do
    out=${f/.srt.fil.bed/.intron.bed}
    bedtools intersect -S -wo -a $f -b genomes/ws253/ws253.intron.filter.bed > $out
done

# Done